<a id='TOC'></a>

# Cleaning Data (Part 1)
The purpose of this notebook is to read in raw excel data for multiple years, rename and trim columns, append cleaned files into a single dataframe and export this dataframe as an excel file.

# Table of Contents
1. [Setting up Python](#SettingUp)
    
    1. [Setting the Location](#SettingLoc)
    
    2. [Importing Necessary Packages](#ImportingPackages)
    
    3. [Functions](#functions)
    
    4. [Preparing for a Save](#PreparingSave)  

2. [Handling Columns](#HandlingColumns)
    
    1. [Find Unique Column Names](#FindUniqueCol)
    
    2. [Eliminate Unnecessary Columns](#DropCol)
    
    3. [Combine Synonyms](#CombineCol)

3. [Reading and Appending Data](#ReadingAppendingData)

4. [Exporting Data](#ExportingData)

<a id='SettingUp'></a>

# Setting up Python
[Top](#TOC)

[Setting the Location](#SettingLoc)
    
[Importing Necessary Packages](#ImportingPackages)
    
[Getting Data](#GettingData)
    
[Preparing for a Save](#PreparingSave)

<a id='ImportingPackages'></a>

## Importing Necessary Packages

[Top](#TOC)

[Setting Up Python](#SettingUp)

Here we import necessary packages. 
This chunk may take a while.

In [1]:
import pandas as pd
# import numpy as np
import glob,os

# increase print limit
pd.options.display.max_rows = 99999
pd.options.display.max_colwidth = 50
pd.set_option('mode.sim_interactive', True)

<a id='functions'></a>

## Functions
[Back to: Top](#TOC)

[Back to: Setting Up Python](#SettingUp)

1. [xlcolshape](#xlcolshape)

2. [xluniquecol2](#xluniquecol2)

3. [colmatchtodict](#colmatchtodict)

4. [findsyn](#findsyn)

5. [readnsplit](#readnsplit)

6. [mapndrop](#mapndrop)

7. [namefile](#namefile)

<a id='xlcolshape'></a>

[Back to: Functions](#Functions)

In [2]:
def xlcolshape(file, verbose = True):
    """xlcolshape takes a file name as a string and returns the shape of the excel file"""
    assert isinstance(verbose,bool),"'verbose' must be bool not,{}".format(type(verbose))
    dictionary = {}
    for sheet in pd.ExcelFile(file).sheet_names:
        try:
            tmp = pd.read_excel(file,sheet_name =sheet).shape
            dictentry = file+'_'+sheet
            dictionary[dictentry] = tmp
            if verbose == True:
                print("Doing stuff you asked me to do for file \'{}\',sheet \'{}\' programmer person."\
                      .format(file, sheet))            
        except:
            print("This didn't work for file {}, sheet {}".format(file, sheet))
            
    return dictionary

<a id='xluniquecol2'></a>

[Back to: Functions](#Functions)

In [3]:
def xluniquecol2(file, header = 0, verbose=True):
    tmp = []
    for sheet in pd.ExcelFile(file).sheet_names:
        if (('species' in pd.read_excel(file,sheet_name=sheet, header = header).columns)\
            or('Species' in pd.read_excel(file,sheet_name=sheet, header = header).columns)):
            try:
                tmp = list(set(tmp+list(pd.read_excel(file,sheet_name=sheet).columns)))
                if verbose==True:
                    print("Doing stuff you asked me to do for file \'{}\',sheet \'{}\' programmer person."\
                          .format(file,sheet))
                res = tmp
            except:
                print("This didn't work for file {}, sheet {}".format(file,sheet))
        else:
            print("Check columns for file {}.".format(file))
            res = None
    return res
            

<a id='colmatchtodict'></a>

[Back to: Functions](#Functions)

In [4]:
def colmatchtodict(x,series, dictsource, key= None):
    """This takes a string, x, and a looks for values in a series that match that contain that string.
    Those values which match are returned as values in a python dict for the key, key.""" 
    assert isinstance(series,pd.Series)
    if key is None:
        key = x
    tmp = series[series.astype(str).str.contains(x,case = False)].tolist()
    dictsource[key] = tmp
    return dictsource
    

<a id='findsyn'></a>

[Back to: Functions](#Functions)

In [5]:
def findsyn (name,dictionary, verbose = True):
    """
    *findsyn* checks searches the values of the dict *dictionary* for the string, *name* and returns 
    the key for the key,value pair to which *name* belongs.
    """
    tmp = pd.DataFrame({'preferredcol':list(dictionary.keys()),'synonymns':list(dictionary.values())})
    try:
        res = list(tmp.preferredcol[tmp.synonymns.apply(lambda x:name in x)])[0]
    except:
        res = None
        if verbose == True:
            print("No value matching \"{}\" was found in the dictionary.".format(name))
    return res


<a id='readnsplit'></a>

[Back to: Functions](#Functions)

In [6]:
def readnsplit(file,newsourcefolder,dtype=None,verbose=True):
    """
    This function reads an excel file, splits its sheets into separate files and saves them to folder
    *newsourcefolder*.
    """
    suffix = '.'+file.split('.')[1]
    prefix = file[:-len(suffix)]
    for sheet in pd.ExcelFile(file).sheet_names:
        try:
            splitfile = newsourcefolder+'/'+prefix+'_'+sheet+suffix
            tmp = pd.read_excel(file,dtype=dtype, sheet_name=sheet).to_excel(splitfile,index=False)
            if verbose==True:
                print("Succes!  \'{}\',sheet \'{}\' has been saved to {} and the corresponding\
                google drive file as {}.".format(file,sheet,newsourcefolder,splitfile))
            continue
        except:
            print("Unable to save \'{}\',sheet \'{}\' as a separate file.".format(file,sheet))         


<a id='mapndrop'></a>

[Back to: Functions](#Functions)

In [7]:
def mapndrop(df,dictionary,verbose=True):
    """
    This function renames columns in *df* deemed synonymous according to a dict,
    *dictionary*, and drops unnecessary columns before returning the cleaner dataframe.
    """
    try:
        df.columns = pd.Series(df.columns).map(lambda x:dictionary[x])
        tmp = df
        if verbose==True:
            print("Successfully mapped columns for df.")
        dropidx =[None==col for col in list(tmp.columns)]
        tmp=tmp.drop(columns=df.columns[dropidx])
        if verbose==True:
            print("Successfully dropped unnecessary columns for df.")
    except:
        tmp = None
        print("Skipping mapndrop call for df.")
    return tmp


<a id='namefile'></a>

[Back to: Functions](#Functions)

In [8]:
def namefile(name, tzadjust=5,tzdirection = '-', adjprecision='minutes', filetype = 'csv'):
    """takes a filename and filetype, and adds a timestamp adjusted relative to gmt to a precision 
    and returns a string that concatenates them."""
    assert isinstance(name,str),"'name' must be of type str."
    assert isinstance(tzadjust,int),"'tzadjust' must be of type int"
    assert adjprecision in ['date','hour','minutes','seconds', 'max'], "'adjprecision' must be either \
    'date', hour','minutes','seconds', or 'max'"
    precision= {'max':None,'seconds':-7,'minutes':-9, 'hours':-14,'date':-20}
    if tzdirection== '-':
        timestamp = (pd.to_datetime('now')-pd.Timedelta(hours=tzadjust))
    else:
        timestamp = (pd.to_datetime('now')+pd.Timedelta(hours=int(tzadjust[1:])))
    timestamp = str(timestamp).replace(':','hrs',1).replace(':','min',1)
    timestamp = timestamp[:precision[adjprecision]]
    filename = name+'_' + timestamp+ '.' +filetype
    return filename


<a id='PreparingSave'></a>

## Preparing for a Save
[Top](#TOC)

[Setting up Python](#SettingUp)

<a id='SettingLoc'></a>

## Setting the Location
[Top](#TOC)

[Setting Up Python](#SettingUp)

These chunks identify the locations from which we can get data and to which we can save data.

### Source Data
Raw data can be found in the following locations:

In [9]:
# sourceDataPers = 'C:/Users/Christopher/Google Drive/TailDemography/outputFiles'
sourceDataBig = 'S:/Chris/TailDemography/TailDemography/Raw Data'
# sourceBlack = 'C:/Users/test/Desktop'

### Intermediate Source Data
Intermediate files can be found in the following locations:

In [10]:
# sourceInterDataPers = 'C:/Users/Christopher/Google Drive/TailDemography/Intermediate Files/Source'
sourceinterDataBig = 'S:/Chris/TailDemography/TailDemography/Intermediate Files/Source'
# sourceBlack = 'C:/Users/test/Desktop'

Now we change the working directory to the source path.

In [11]:
os.chdir(sourceDataBig)

### Output Data
The cleaned data will be saved to one of these locations:

In [12]:
# outputPers = 'C:/Users/Christopher/Google Drive/TailDemography/outputFiles'
outputBig = 'S:/Chris/TailDemography/TailDemography/Cleaned Combined Data'
# outputBlack = 'C:/Users/test/Desktop'

### Review files

In [13]:
# outputPers = 'C:/Users/Christopher/Google Drive/TailDemography/Files for review/Source files'
reviewfolderBig = 'S:/Chris/TailDemography/TailDemography/Files for review/Source files'

<a id='HandlingColumns'></a>

# Handling Columns
[Top](#TOC)

We don't have to look in the multiple-sheet file.  It's clear that we'll have to identify a common set of columns prior to combining these files.  Let's define a few functions to help us do this.

We will want to do the following:
1. [Find Unique Column Names](#FindUniqueCol)
2. [Eliminate Unnecessary Columns](#DropCol)
3. [Combine Synonyms](#CombineCol)

Here we use search the source path to locate and eventually read the raw data into our notebook.

In [14]:
rawfiles = glob.glob('*.xls*')
rawfiles

['CC 2000-03-modified from CC-SJ 00-03 final-modified w headers-3Jan19.xlsx',
 'CC 2004.xlsx',
 'CC 2015 - captures.xls',
 'CC 2016 - captures.xls',
 'CC 2017 Lizards - 3viii17 captures and obs.xls',
 'xCC2005x.xls',
 'xCC2006x.xls',
 'xCC2007x.xls',
 'xCC2008x.xls',
 'xCC2009x.xls',
 'xCC2010x.xlsx',
 'xCC2011x.xls',
 'xCC2012x.xls',
 'xCC2013x.xls',
 'xCC2014x.xlsx']

We'll separate these into files with single or multiple sheets.

In [15]:
rawfiles_ms = [rawfiles[0],rawfiles[7]]
rawfiles_ss = list(set(rawfiles)- set(rawfiles_ms))

The names of files with multiple sheets are now in the variable *rawfiles_ms*.

In [16]:
rawfiles_ms

['CC 2000-03-modified from CC-SJ 00-03 final-modified w headers-3Jan19.xlsx',
 'xCC2007x.xls']

The names of files with a single sheet are now in the variable *rawfiles_ss*.

In [17]:
rawfiles_ss

['xCC2008x.xls',
 'CC 2016 - captures.xls',
 'CC 2004.xlsx',
 'xCC2005x.xls',
 'xCC2010x.xlsx',
 'xCC2011x.xls',
 'xCC2013x.xls',
 'xCC2014x.xlsx',
 'CC 2017 Lizards - 3viii17 captures and obs.xls',
 'xCC2012x.xls',
 'CC 2015 - captures.xls',
 'xCC2009x.xls',
 'xCC2006x.xls']

Now let's take a look at the number of columns in each file. We'll start with the single sheet files, since this is the easiest.  We will use the function, *xlcolshape* to make this easier. 
When we call this function on the first of the single-sheet files, we can see that it returns a tuple in the format ('number of rows', 'number of columns'). The code for *xlcolshape* can be found in [Functions](#functions).

In [18]:
xlcolshape(rawfiles_ss[0])

Doing stuff you asked me to do for file 'xCC2008x.xls',sheet '2008' programmer person.


{'xCC2008x.xls_2008': (134, 20)}

We will apply this function to the list of files for our inspection.

In [19]:
pd.Series(rawfiles).apply(lambda x: xlcolshape(x,verbose=False))

0     {'CC 2000-03-modified from CC-SJ 00-03 final-m...
1                     {'CC 2004.xlsx_2004 ': (479, 16)}
2            {'CC 2015 - captures.xls_2015': (241, 19)}
3            {'CC 2016 - captures.xls_2016': (103, 21)}
4     {'CC 2017 Lizards - 3viii17 captures and obs.x...
5                      {'xCC2005x.xls_2005': (202, 16)}
6                      {'xCC2006x.xls_2006': (163, 16)}
7     {'xCC2007x.xls_Sheet1': (507, 16), 'xCC2007x.x...
8                      {'xCC2008x.xls_2008': (134, 20)}
9                      {'xCC2009x.xls_2009': (162, 16)}
10                   {'xCC2010x.xlsx_Sheet1': (99, 41)}
11                    {'xCC2011x.xls_Sheet1': (64, 19)}
12                      {'xCC2012x.xls_data': (85, 19)}
13             {'xCC2013x.xls_CC 2013 data': (106, 19)}
14                     {'xCC2014x.xlsx_2014': (97, 19)}
dtype: object

<a id='FindUniqueCol'></a>

## Finding Unique Columns
[Top](#TOC)

[Handling Columns](#HandlingColumns)

We'll use the function, *xluniqucol2* to extract column names and convert them to an approved set.  We'll use that function to allow us to only add unique names to a list of column names. 

Here is an example of how xluniquecol2 works for a file with one sheet.  You can find the code for *xluniquecol2* in [Functions](#functions).

In [20]:
xluniquecol2(rawfiles_ss[0],verbose=False)

['Location',
 'Sex',
 'RTL',
 'Meters',
 'Time',
 'New/Recap',
 'painted',
 'Date',
 'Toes',
 'SVL',
 'Mass',
 'Misc.',
 'Marked',
 'Vial',
 'Unnamed: 19',
 'Paint Mark',
 'Spotted',
 'Unnamed: 16',
 'TL',
 'Species']

Here is an example of how xluniquecol2 works for a file with multiple sheets.

In [21]:
xluniquecol2(rawfiles_ms[0],verbose=False)

['painted or not',
 'RTL',
 'date',
 'TOES',
 'meters',
 'NEW/recap',
 'SVL',
 'mass',
 'VIAL',
 'TIME',
 'sex',
 'paint mark',
 'location',
 'misc',
 'Unnamed: 16',
 'TL',
 'species']

Now we will create an empty set, *uniquecols2*, that will eventually contain the unique column names in all of the files.

We will append the unique column names from each file to *uniquecols2*.

In [22]:
tmp = pd.Series(rawfiles).apply(xluniquecol2,verbose=False)
uniquecols2 = list()
for u in tmp:
    uniquecols2 = uniquecols2+u
uniquecols2 = list(set(uniquecols2))
uniquecols2

['Sex',
 'Meters',
 1,
 'New/Recap',
 'Toe 6',
 'Toes',
 'Collectors',
 'Misc.',
 'Unnamed: 19',
 'sex',
 'Unnamed: 17',
 'paint mark',
 'misc',
 'Spotted',
 'Toe 5',
 'mass (g)',
 'SVL (mm)',
 'misc/notes',
 'RTL',
 'Painted',
 'Toe 9',
 'Years Alive (known)',
 'Tail condition (1=intact; 2=autotomized; 3=regrown)',
 'Toe 13',
 'location',
 'Mark',
 'Unnamed: 16',
 'Toe 11',
 'Location',
 'Time',
 'Toe 7',
 'meters',
 'Painted or Not',
 'Toe 14',
 ' painted or not',
 'Toe 16',
 'Toe 12',
 'SVL',
 'TIME',
 'Year',
 'Toe 15',
 'Vial',
 '1st Capture (year)',
 '2015 or earlier',
 'RTL (mm)',
 'TL (mm)',
 'TL',
 'Paint Mark',
 'Toe 18',
 'Toe 1',
 'painted or not',
 'date',
 'TOES',
 'Toe 8',
 'NEW/recap',
 'Toe 4',
 'Marked',
 'painted',
 'Date',
 'Mass',
 'Toe 19',
 'mass',
 'VIAL',
 'Toe 2',
 'Toe 20',
 'Toe 17',
 'Toe 10',
 'Toe 3',
 'species',
 'Species']

<a id='DropCol'></a>

## Eliminate Unnecessary Columns
[Top](#TOC)

[Cleaning Data](#CleaningData)

[Handling Columns](#HandlingColumns)

Now we will try to identify unnecessary columns and eliminate them. Much of this will be done manually.

In [23]:
keepCol = ['species', 'date', 'sex', 'svl', 'tl', 'rtl', 'mass',
       'paint.mark', 'location', 'meters', 'new.recap', 'painted', 'misc',
       'vial', 'autotomized', 'sighting', 'toes','filename']

In [24]:
set(pd.Series(keepCol).str.lower())-set(pd.Series(uniquecols2).str.lower())

{'autotomized', 'filename', 'new.recap', 'paint.mark', 'sighting'}

In [25]:
set(pd.Series(uniquecols2).str.lower())-set(pd.Series(keepCol).str.lower())

{' painted or not',
 '1st capture (year)',
 '2015 or earlier',
 'collectors',
 'mark',
 'marked',
 'mass (g)',
 'misc.',
 'misc/notes',
 nan,
 'new/recap',
 'paint mark',
 'painted or not',
 'rtl (mm)',
 'spotted',
 'svl (mm)',
 'tail condition (1=intact; 2=autotomized; 3=regrown)',
 'time',
 'tl (mm)',
 'toe 1',
 'toe 10',
 'toe 11',
 'toe 12',
 'toe 13',
 'toe 14',
 'toe 15',
 'toe 16',
 'toe 17',
 'toe 18',
 'toe 19',
 'toe 2',
 'toe 20',
 'toe 3',
 'toe 4',
 'toe 5',
 'toe 6',
 'toe 7',
 'toe 8',
 'toe 9',
 'unnamed: 16',
 'unnamed: 17',
 'unnamed: 19',
 'year',
 'years alive (known)'}

Since data for years 2000-2003 are contained in the same Excel file we will have to treat this file differently than the others.

<a id='CombineCol'></a>

## Combining Synonymous Columns
[Top](#TOC)

[Cleaning Data](#CleaningData)

[Handling Columns](#HandlingColumns)

Once we have identified the columns we need to keep, we'll need to apply this list to the files as they are read into python by doing the following:

We will use a function, *colmatchtodict*,  to identify potential synonyms. Here's an example of how *colmatchtodict* works.  The code for this function can be found in [Functions](#functions).

In [26]:
coldict = {}

In [27]:
colmatchtodict('toes',pd.Series(uniquecols2),coldict, key = 'toes')

{'toes': ['Toes', 'TOES']}

Now let's see what happened when we apply this funtion to our, keepCol.

In [28]:
coldict = {}

In [29]:
pd.Series(keepCol).apply(lambda x: colmatchtodict(x=x,series=pd.Series(uniquecols2),dictsource=coldict))
coldict

{'species': ['species', 'Species'],
 'date': ['date', 'Date'],
 'sex': ['Sex', 'sex'],
 'svl': ['SVL (mm)', 'SVL'],
 'tl': ['RTL', 'RTL (mm)', 'TL (mm)', 'TL'],
 'rtl': ['RTL', 'RTL (mm)'],
 'mass': ['mass (g)', 'Mass', 'mass'],
 'paint.mark': ['paint mark', 'Paint Mark'],
 'location': ['location', 'Location'],
 'meters': ['Meters', 'meters'],
 'new.recap': ['New/Recap', 'NEW/recap'],
 'painted': ['Painted',
  'Painted or Not',
  ' painted or not',
  'painted or not',
  'painted'],
 'misc': ['Misc.', 'misc', 'misc/notes'],
 'vial': ['Vial', 'VIAL'],
 'autotomized': ['Tail condition (1=intact; 2=autotomized; 3=regrown)'],
 'sighting': [],
 'toes': ['Toes', 'TOES'],
 'filename': []}

We will manually adjust the values for 'tl' and 'filename'.

In [30]:
coldict['tl']=['TL (mm)', 'TL', 'tl']

Now we need to use this dict to relabel the columns we wish to keep.

We will use the function, *findsyn* to identify potential synonymous to the columnlabels in *keepcols* among the column labels in *uniquecols2*. 

Here is are a few examples of how *findsyn* works.  The code can be found in [Functions](#functions).

In [31]:
findsyn('RTi',coldict,verbose=False)

In [32]:
findsyn('RTi',coldict,verbose=True)

No value matching "RTi" was found in the dictionary.


In [33]:
findsyn('RTL',coldict,verbose=True)

'rtl'

Now we apply *findsyn* to *uniquecol* and create a column of synonyms.

In [34]:
uniquecols2df = pd.DataFrame({'uniquecols2':uniquecols2})
uniquecols2df['preferredcol'] = uniquecols2df.uniquecols2.apply(lambda x: findsyn(x,coldict,False))
uniquecols2df

,uniquecols2,preferredcol
0,Sex,sex
1,Meters,meters
2,1,None
3,New/Recap,new.recap
4,Toe 6,None
5,Toes,toes
6,Collectors,None
7,Misc.,misc
8,Unnamed: 19,None
9,sex,sex


Now we will turn this dataframe back into a dict so that we can easily use it to rename columns

In [35]:
uniquecols2df.index = uniquecols2df.uniquecols2
uniquecols2dict = pd.Series(uniquecols2df.preferredcol).to_dict()
uniquecols2dict

{'Sex': 'sex',
 'Meters': 'meters',
 1: None,
 'New/Recap': 'new.recap',
 'Toe 6': None,
 'Toes': 'toes',
 'Collectors': None,
 'Misc.': 'misc',
 'Unnamed: 19': None,
 'sex': 'sex',
 'Unnamed: 17': None,
 'paint mark': 'paint.mark',
 'misc': 'misc',
 'Spotted': None,
 'Toe 5': None,
 'mass (g)': 'mass',
 'SVL (mm)': 'svl',
 'misc/notes': 'misc',
 'RTL': 'rtl',
 'Painted': 'painted',
 'Toe 9': None,
 'Years Alive (known)': None,
 'Tail condition (1=intact; 2=autotomized; 3=regrown)': 'autotomized',
 'Toe 13': None,
 'location': 'location',
 'Mark': None,
 'Unnamed: 16': None,
 'Toe 11': None,
 'Location': 'location',
 'Time': None,
 'Toe 7': None,
 'meters': 'meters',
 'Painted or Not': 'painted',
 'Toe 14': None,
 ' painted or not': 'painted',
 'Toe 16': None,
 'Toe 12': None,
 'SVL': 'svl',
 'TIME': None,
 'Year': None,
 'Toe 15': None,
 'Vial': 'vial',
 '1st Capture (year)': None,
 '2015 or earlier': None,
 'RTL (mm)': 'rtl',
 'TL (mm)': 'tl',
 'TL': 'tl',
 'Paint Mark': 'paint.mark'

We'll use the dict, *uniquecols2dict* to rename the synonymous columns in our file....once we read them in,
that is.

<a id='ReadingAppendingData'></a>

# Reading and Appending Data
[Top](#TOC)

Now we use the function *readnsplit* to actually read in the source files, drop unnecessary columns and renaming columns according to a dictionary. 

Here is an example of how *readnsplit* works.  The code can be found in [Functions](#functions).

In [36]:
readnsplit(rawfiles[0],sourceinterDataBig,str)

Succes!  'CC 2000-03-modified from CC-SJ 00-03 final-modified w headers-3Jan19.xlsx',sheet '2000' has been saved to S:/Chris/TailDemography/TailDemography/Intermediate Files/Source and the corresponding                google drive file as S:/Chris/TailDemography/TailDemography/Intermediate Files/Source/CC 2000-03-modified from CC-SJ 00-03 final-modified w headers-3Jan19_2000.xlsx.
Succes!  'CC 2000-03-modified from CC-SJ 00-03 final-modified w headers-3Jan19.xlsx',sheet '2001' has been saved to S:/Chris/TailDemography/TailDemography/Intermediate Files/Source and the corresponding                google drive file as S:/Chris/TailDemography/TailDemography/Intermediate Files/Source/CC 2000-03-modified from CC-SJ 00-03 final-modified w headers-3Jan19_2001.xlsx.
Succes!  'CC 2000-03-modified from CC-SJ 00-03 final-modified w headers-3Jan19.xlsx',sheet '2002' has been saved to S:/Chris/TailDemography/TailDemography/Intermediate Files/Source and the corresponding                google drive f

In [37]:
for file in rawfiles:
    readnsplit(file,sourceinterDataBig,dtype=str, verbose=False)

We need to change the directory to the location where the intermediate files this operates on can be found.  We will also save a list of the files names in that location for convenience.

In [38]:
os.chdir(sourceinterDataBig)
splitfiles = glob.glob('*xls*')
splitfiles

['CC 2000-03-modified from CC-SJ 00-03 final-modified w headers-3Jan19_2000.xlsx',
 'CC 2000-03-modified from CC-SJ 00-03 final-modified w headers-3Jan19_2001.xlsx',
 'CC 2000-03-modified from CC-SJ 00-03 final-modified w headers-3Jan19_2002.xlsx',
 'CC 2000-03-modified from CC-SJ 00-03 final-modified w headers-3Jan19_2003.xlsx',
 'CC 2004_2004 .xlsx',
 'CC 2015 - captures_2015.xls',
 'CC 2016 - captures_2016.xls',
 'CC 2017 Lizards - 3viii17 captures and obs_2017.xls',
 'xCC2005x_2005.xls',
 'xCC2006x_2006.xls',
 'xCC2007x_2007.xls',
 'xCC2007x_Sheet1.xls',
 'xCC2008x_2008.xls',
 'xCC2009x_2009.xls',
 'xCC2010x_Sheet1.xlsx',
 'xCC2011x_Sheet1.xls',
 'xCC2012x_data.xls',
 'xCC2013x_CC 2013 data.xls',
 'xCC2014x_2014.xlsx']

Now we remove 'xCC2007x_Sheet1.xls' from the list of files we will process intermediate files since this is a subset of the 'xCC2007x_2007.xls' reordered and with some columns dropped.

In [39]:
splitfiles = list(set(splitfiles)-set(['xCC2007x_Sheet1.xls']))
splitfiles

['CC 2000-03-modified from CC-SJ 00-03 final-modified w headers-3Jan19_2002.xlsx',
 'CC 2004_2004 .xlsx',
 'xCC2009x_2009.xls',
 'xCC2011x_Sheet1.xls',
 'xCC2010x_Sheet1.xlsx',
 'xCC2014x_2014.xlsx',
 'CC 2000-03-modified from CC-SJ 00-03 final-modified w headers-3Jan19_2001.xlsx',
 'CC 2017 Lizards - 3viii17 captures and obs_2017.xls',
 'xCC2007x_2007.xls',
 'xCC2008x_2008.xls',
 'CC 2016 - captures_2016.xls',
 'xCC2006x_2006.xls',
 'xCC2013x_CC 2013 data.xls',
 'CC 2000-03-modified from CC-SJ 00-03 final-modified w headers-3Jan19_2000.xlsx',
 'xCC2012x_data.xls',
 'xCC2005x_2005.xls',
 'CC 2000-03-modified from CC-SJ 00-03 final-modified w headers-3Jan19_2003.xlsx',
 'CC 2015 - captures_2015.xls']

Now we use the function *mapndrop* to drop unnecessary columns and renaming columns according to a dictionary.

Here are a few examples of how *mapndrop* works.  The code can be found in [Functions](#functions).

In [40]:
mapndrop(df=pd.read_excel(splitfiles[0],dtype=str),dictionary=uniquecols2dict,verbose = True)

Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.


,species,toes,date,sex,svl,tl,rtl,mass,paint.mark,location,meters,new.recap,painted,misc,vial
0,sj,3-7-11-19,2002-07-14 00:00:00,m,63,92,0,10,b101t,halfway up to site,-200,NEW,nan,nan,toes in vial 58-02
1,sj,3-7-11-18,2002-07-14 00:00:00,m,66,92,0,10.8,b102t,left downstream 100m v 1 falls,-100,NEW,nan,nan,toes in vial 59-02
2,sj,3-7-12-16,2002-07-14 00:00:00,m,68,103,0,10.3,b103t,90m v 1 falls,-90,NEW,nan,nan,toes in vial 60-02
3,sv,nan,2002-07-05 00:00:00,nan,nan,nan,nan,nan,w2a,sb at intersection with trail v 1 falls,-30,sighting,nan,nan,nan
4,sj,10,2002-07-14 00:00:00,m,85,118,0,19.5,b104t,sb - trail intersection v 1 falls,-20,recap,toe loss may be natural,nan,nan
5,sj,nan,2002-07-11 00:00:00,nan,nan,nan,nan,nan,y55b,sb 20m v 1 falls,-20,sighting,"G -, -",nan,nan
6,sv,10-16,2002-07-03 00:00:00,f,63,49,21,11.6,w1a,left side @ base juniper 8 m ^ sb; 15m v 1 falls,-15,NEW,painted; gravid; <5 mites in pockets,nan,toes in vial 34-02 (10-16)
7,sv,10-18,2002-07-03 00:00:00,m,51,73,0,4.8,w2a,left side @ base juniper 8 m ^ sb; 15m v 1 falls,-15,NEW,painted; <5 mites in pockets,nan,nan
8,sj,nan,2002-05-09 00:00:00,nan,nan,nan,nan,nan,w67a,4m v I falls on juniper tree next to cliff onleft,-10,sighting,nan,nan,nan
9,sj,nan,2002-07-04 00:00:00,nan,nan,nan,nan,nan,juv,sb v 1 falls,-10,sighting,not marked because below site,nan,nan


In [41]:
mapndrop(df=pd.read_excel(splitfiles[4],dtype=str),dictionary=uniquecols2dict, verbose=True)

Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.


,species,toes,date,sex,svl,tl,rtl,autotomized,mass,paint.mark,location,meters,new.recap,painted,misc,vial
0,sj,2-6-12-15,19vi2010,m,80,110,29,3,20,y2c,10m v bottom bowl,-15,?,yes,toe 15 missing at capture; possible recap,03-10-cc
1,sj,2-9-15-17,13viii2010,f,56,77,0,1,5.5,y62c,20m up CCC,240,NEW,yes,Tss,61-10-cc
2,sj,3-6-11-17,18viii2010,m,50,68,0,1,4,y<c.t,1m vT at top R island,157,NEW,yes,Bss; lost toes,nan
3,sj,3-6-15-16,18viii2010,f,72,62,46,3,11,y65c,halfway between pool and 2 falls 2m up rt side,385,NEW,yes,nan,65-10-cc
4,sj,3-6-12-17,18viii2010,m,57,82,0,1,6,y67c,R outcrop ^ oak R,425,new,yes,nan,66-10-cc
5,sj,3-6-15-17,18viii2010,m,52,70,0,2,4.5,y68c,r outcrop ^ oak R,425,new,yes,Tss,67-10-cc
6,sj,3-6-15-18,18viii2010,f,53,72,0,1,5,y69c,pine R,408,new,yes,nan,68-10-cc
7,sj,3-6-15-19,18viii2010,f,74,100,0,1,12.5,y70c,5m v wall v wall v juniper xing,80,new,yes,too easy to pass up,69-10-cc
8,sj,2-6-13-16,19viii2010,m,55,74,0,1,5,y71c,3m ^ 1 falls,3,new,yes,nan,70-10-cc
9,sj,2-6-14-17,19viii2010,m,47,64,0,1,3.5,y÷c.t,3m ^ 1 falls,3,new,yes,possible Bss; mark is a yellow division symbol...,71-10-cc


We'll create a df, *df*, with no data, but columns from our desired columns, *i.e.* the keys for coldict, as a placeholder to which we can append new data.

In [42]:
df = pd.DataFrame(columns=coldict.keys())
df

,species,date,sex,svl,tl,rtl,mass,paint.mark,location,meters,new.recap,painted,misc,vial,autotomized,sighting,toes,filename


Now we will read in all of the successfully split files, clean the column names, and concatenate them into one large df.

In [43]:
for file in splitfiles:
    df = pd.concat([df,mapndrop(pd.read_excel(file,dtype=str),uniquecols2dict)],sort=True)
    print(df.shape[0])
print("\n\nFinal df has {} columns and {} rows.".format(df.shape[1],df.shape[0]))
df.head()

Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.
1477
Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.
1956
Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.
2118
Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.
2182
Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.
2281
Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.
2378
Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.
3059
Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.
3857
Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.
4039
Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.
4173
Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.
4276
Successful

,autotomized,date,filename,location,mass,meters,misc,new.recap,paint.mark,painted,rtl,sex,sighting,species,svl,tl,toes,vial
0,NaN,2002-07-14 00:00:00,NaN,halfway up to site,10,-200,nan,NEW,b101t,nan,0,m,NaN,sj,63,92,3-7-11-19,toes in vial 58-02
1,NaN,2002-07-14 00:00:00,NaN,left downstream 100m v 1 falls,10.8,-100,nan,NEW,b102t,nan,0,m,NaN,sj,66,92,3-7-11-18,toes in vial 59-02
2,NaN,2002-07-14 00:00:00,NaN,90m v 1 falls,10.3,-90,nan,NEW,b103t,nan,0,m,NaN,sj,68,103,3-7-12-16,toes in vial 60-02
3,NaN,2002-07-05 00:00:00,NaN,sb at intersection with trail v 1 falls,nan,-30,nan,sighting,w2a,nan,nan,nan,NaN,sv,nan,nan,nan,nan
4,NaN,2002-07-14 00:00:00,NaN,sb - trail intersection v 1 falls,19.5,-20,nan,recap,b104t,toe loss may be natural,0,m,NaN,sj,85,118,10,nan


In [44]:
df = df.reindex(['species', 'toes', 'sex', 'date', 'svl', 'tl', 'rtl', 'autotomized', 'mass', 
                 'location', 'meters', 'new.recap', 'painted', 'sighting', 
                 'paint.mark', 'vial', 'misc'], axis=1)
df.head()

,species,toes,sex,date,svl,tl,rtl,autotomized,mass,location,meters,new.recap,painted,sighting,paint.mark,vial,misc
0,sj,3-7-11-19,m,2002-07-14 00:00:00,63,92,0,NaN,10,halfway up to site,-200,NEW,nan,NaN,b101t,toes in vial 58-02,nan
1,sj,3-7-11-18,m,2002-07-14 00:00:00,66,92,0,NaN,10.8,left downstream 100m v 1 falls,-100,NEW,nan,NaN,b102t,toes in vial 59-02,nan
2,sj,3-7-12-16,m,2002-07-14 00:00:00,68,103,0,NaN,10.3,90m v 1 falls,-90,NEW,nan,NaN,b103t,toes in vial 60-02,nan
3,sv,nan,nan,2002-07-05 00:00:00,nan,nan,nan,NaN,nan,sb at intersection with trail v 1 falls,-30,sighting,nan,NaN,w2a,nan,nan
4,sj,10,m,2002-07-14 00:00:00,85,118,0,NaN,19.5,sb - trail intersection v 1 falls,-20,recap,toe loss may be natural,NaN,b104t,nan,nan


In [45]:
df.shape

(6299, 17)

<a id='ExportingData'></a>

# Exporting Data
[Top](#TOC)

Here we call the function, *namefile*, to create a timestamped name for file to be exported.  You can find the code for *namefile* in [Functions](#functions).

In [46]:
filename = namefile('Appended and Trimmed CC Data 2000-2017')
os.chdir(outputBig)
df.to_csv(filename,index = False)
print("\'{}\' has been saved to \'{}\' and the corresponding drive google drive location."\
      .format(filename, outputBig))

'Appended and Trimmed CC Data 2000-2017_2019-03-10 13hrs38min.csv' has been saved to 'S:/Chris/TailDemography/TailDemography/Cleaned Combined Data' and the corresponding drive google drive location.
